# Question 2 Workflow


## Workfolow Summary

## Imports

In [1]:
# Import pandas and biothings explorers modules
import pandas as pandas
from biothings_explorer.query.predict import Predict
from biothings_explorer.query.visualize import display_graph
from biothings_explorer.user_query_dispatcher import FindConnection
from biothings_explorer.hint import Hint
import nest_asyncio
nest_asyncio.apply()
%matplotlib inline
import warnings
warnings.filterwarnings("ignore") 

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/data_tools/df_processing.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Set Parameters

In [2]:
max_direct = 20
max_indirect = 50

In [3]:
disease_name = 'COVID-19'
ht = Hint()
disease = ht.query(disease_name)['Disease'][0]
disease

{'MONDO': 'MONDO:0100096',
 'DOID': 'DOID:0080600',
 'name': 'COVID-19',
 'primary': {'identifier': 'MONDO',
  'cls': 'Disease',
  'value': 'MONDO:0100096'},
 'display': 'MONDO(MONDO:0100096) DOID(DOID:0080600) name(COVID-19)',
 'type': 'Disease'}

## Get Symptoms

In [4]:
import requests 

symptoms = ([ 'fever','cough', 'hemoptysis', 
'shortness of breath', 'myalgia', 'fatigue', 'sore throat', 
'nausea', 'vomiting', 'diarrhea', 'conjunctivitis', 'anorexia',
'headache', 'leukopenia', 'eosinopenia', 'neutrophilia', 'elevated liver enzymes', 'C-reactive protein', 'ferritin',
'hyaline membrane', 'alveolar lesions', 'prominent hyaline membranes', 'alveolar damage',
 'acute myocardial injury', ' myocarditis', 'arrhythmias','cardiac dysfunction', 'encephalopathy'])


symptom_dict = {}

for symptom_name in symptoms: 
    try: 
        symptom = ht.query(symptom_name)['PhenotypicFeature'][0]
#         print(symptom['name'])
#         pd = Predict(
#             input_objs=[symptom],
#             intermediate_nodes =[None], 
#             output_types = ['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 
#                 'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 
#                 'Pathway', 'AnatomicalEntity', 'PhenotypicFeature'], 
#             config={"expand": False}
#         )
#         pd.connect(verbose=False)
#         df = pd.display_table_view()
#         edges_out = df.shape[0]
        edges_out = 10
        symptom_dict[symptom['HP']] = {
            'names': [symptom['name'].upper()],
            'frequency': 'Unknown', 
            'edges_out' : edges_out
        }
        r = requests.get('https://biothings.ncats.io/hpo/phenotype/' + str(symptom['HP']))
        res = r.json()
#         print(res)
        if('synonym' in res):
            if('exact' in res['synonym']):
                for name in res['synonym']['exact']:
                    if name.upper() not in symptom_dict[res['_id']]['names']: 
#                         print(name.upper())
                        symptom_dict[res['_id']]['names'].append(name.upper())
    except:
        print(symptom_name + " failed")


eosinopenia failed
hyaline membrane failed
alveolar lesions failed
prominent hyaline membranes failed
acute myocardial injury failed
cardiac dysfunction failed


In [5]:
# symptom_dict
disease_symptom_df = pandas.DataFrame.from_dict(symptom_dict, orient='index')
disease_symptom_df

,names,frequency,edges_out
HP:0011134,"[LOW-GRADE FEVER, MILD FEVER]",Unknown,10
HP:0012735,"[COUGH, COUGHING]",Unknown,10
HP:0002105,"[HEMOPTYSIS, COUGHING UP BLOOD, HAEMOPTYSIS]",Unknown,10
HP:0002098,"[RESPIRATORY DISTRESS, BREATHING DIFFICULTIES,...",Unknown,10
HP:0003326,"[MYALGIA, MUSCLE ACHE, MUSCLE PAIN]",Unknown,10
HP:0012378,"[FATIGUE, TIRED, TIREDNESS]",Unknown,10
HP:0033050,"[THROAT PAIN, SORE THROAT]",Unknown,10
HP:0002018,[NAUSEA],Unknown,10
HP:0025089,"[FECULENT VOMITING, FECAL VOMITING, STERCORACE...",Unknown,10
HP:0002014,"[DIARRHEA, DIARRHOEA, WATERY STOOL]",Unknown,10


In [6]:
## also save disease symptoms as a list
disease_symptoms = []
for x in disease_symptom_df['names']: 
    disease_symptoms = disease_symptoms + x
# disease_symptoms

## Disease -> Gene -> [Phenotypic Feature, Disease, Biological Process]

In [7]:
pd = Predict(
    input_objs=[disease],
    intermediate_nodes =['Gene'], 
    output_types =['PhenotypicFeature', 'Disease', 'BiologicalProcess'], 
    config={"expand": False}
)
pd.connect(verbose=False)

API call to Automat CORD19 Scibite API with input 7054 failed with timeout error. Current timeout limit is 5 seconds
API call to CORD Gene API with input ['399', '6018', '5962', '3009', '5414', '6023', '11782', '11876', '11892', '4113', '9281', '5438', '1706', '5417', '16884', '5992', '11183', '6741', '19679', '1116', '10627', '79', '6001', '2367', '338', '13523', '27960', '6025', '2524', '195', '10618', '6915', '30615', '27954', '4552', '11947', '9208', '17697', '6898', '5434', '4989', '10637', '336', '1678', '6008', '8568', '2707', '3535', '5981', '2537', '9958', '13557', '16885', '3541'] failed with timeout error. Current timeout limit is 20 seconds
API call to CORD Gene API with input ['399', '6018', '5962', '3009', '5414', '6023', '11782', '11876', '11892', '4113', '9281', '5438', '1706', '5417', '16884', '5992', '11183', '6741', '19679', '1116', '10627', '79', '6001', '2367', '338', '13523', '27960', '6025', '2524', '195', '10618', '6915', '30615', '27954', '4552', '11947', '9208

In [8]:
df = pd.display_table_view()
df.shape

(32428, 19)

In [9]:
# filter gene -> symptoms table to only include rows where symptoms match disease symptoms
indices_with_symptom_outputs = [i for i, val in enumerate([x.upper() for x in list(df['output_label'])]) if val in disease_symptoms]
direct_relevant_genes_to_symptoms_df = df.iloc[indices_with_symptom_outputs]
direct_relevant_genes_to_symptoms_df.shape

(102, 19)

In [10]:
direct_genes = 	list(direct_relevant_genes_to_symptoms_df["node1_label"])
direct_gene_dict = {x:direct_genes.count(x) for x in direct_genes}
direct_gene_dict = dict(sorted(direct_gene_dict.items(), key=lambda x: x[1], reverse=True)[:max_direct])
direct_gene_dict

{'TNF': 21,
 'CRP': 12,
 'IFNA1': 12,
 'CD4': 7,
 'IL6': 7,
 'IL2': 6,
 'ACE': 4,
 'MARS1': 3,
 'CCL2': 3,
 'F2': 3,
 'IFNG': 2,
 'IL1B': 2,
 'F3': 2,
 'TNNI3': 2,
 'GPT': 2,
 'MB': 2,
 'CD8A': 2,
 'POR': 1,
 'SON': 1,
 'ALB': 1}

## Disease X -> Intermediate Node -> Gene ->  [Phenotypic Feature, Disease, Biological Process]

In [11]:
query_config = {
    "filters": [
        {
        },
        {
            "nodeDegree": {
                ">" : 20
            }
        }
    ],
    "annotate": ["ngd","nodeDegree"],
    "expand": False
}

In [12]:
## way to filter at the gene step for top x number of genes? 
pd2 = Predict(
    input_objs=[disease],
    intermediate_nodes =[['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 
                'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 
                'Pathway', 'AnatomicalEntity', 'PhenotypicFeature'],'Gene'], 
    output_types =['PhenotypicFeature', 'Disease', 'BiologicalProcess'], 
    config= query_config 
)
pd2.connect(verbose=True)

========== QUERY PARAMETER SUMMARY ==========

Your query have 1 input nodes, including COVID-19 .... And BTE will find paths that connect your input nodes to your output types ['PhenotypicFeature', 'Disease', 'BiologicalProcess']. Paths will contain 2 intermediate nodes.

Intermediate node #1 will have these type constraints: ['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 'Pathway', 'AnatomicalEntity', 'PhenotypicFeature']
Intermediate node #2 will have these type constraints: Gene

========== Query # 1 ==========

==== Step #1: Query Path Planning ====

Input Types: Disease
Output Types: ['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 'Pathway', 'AnatomicalEntity', 'PhenotypicFeature']
Predicates: None

BTE found 12 APIs based on SmartAPI Meta-KG.

API 1. Ontology Lookup Service API (1 API calls)
API 2. Automat CORD19 Scibite API (3 AP

API 15.1 MyChem.info API: 21 hits
API 15.2: https://mychem.info/v1/query?fields=drugcentral.bioactivity (POST -d q=CHEMBL729,CHEMBL1690,CHEMBL76,CHEMBL1234886,CHEMBL58510,CHEMBL1669,CHEMBL283800,CHEMBL163,CHEMBL1214598,CHEMBL2095223,CHEMBL1326,CHEMBL47047,CHEMBL1535&scopes=chembl.molecule_chembl_id)
API 15.2 MyChem.info API: 31 hits
API 4.1: https://biothings.ncats.io/tcga_mut_freq_kp/query?fields=association.freq_by_sample,subject.SYMBOL&q=object.id:"MONDO:0020753"%20AND%20association.freq_by_sample:>0.1&size=1000&sort=-association.freq_by_sample
API 4.1 TCGA Mutation Frequency KP API: 0 hits
API 6.1: https://pending.biothings.io/DISEASES/query?fields=DISEASES.associatedWith (POST -d q=DOID:0080599&scopes=DISEASES.doid)
API 6.1 DISEASES API: 0 hits
API 3.1: https://pending.biothings.io/mgigene2phenotype/query?fields=_id&size=300 (POST -d q=DOID:0080599&scopes=mgi.associated_with_disease.doid)
API 3.1 MGIgene2phenotype API: 0 hits
API 11.1: https://mygene.info/v3/query?fields=pantherdb

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 297, in callSingleAPI
    result = tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/__init__.py", line 49, in transform
    return self.tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/base_transformer.py", line 78, in transform
    item = self.wrap(item)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/cord_transformer.py", line 16, in wrap
    for k in item.keys():
RuntimeError: dictionary keys changed during iteration


API 19.3 Automat HMDB API: 0 hits
API 19.4: https://automat.renci.org/hmdb/chemical_substance/gene/CHEBI:25805
API 19.4 Automat HMDB API: 0 hits
API 17.1: https://biothings.ncats.io/cord_chemical/query?fields=associated_with (POST -d q=CHEBI:15379,CHEBI:31781,CHEBI:134730,CHEBI:24261,CHEBI:29194,CHEBI:57947,CHEBI:45409,CHEBI:5801,CHEBI:33098,CHEBI:48433,CHEBI:29995,CHEBI:132943,CHEBI:25805,CHEBI:27140,CHEBI:3638,CHEBI:16919&scopes=chebi)
API 14.3: https://biothings.ncats.io/semmedchemical/query?fields=affects (POST -d q=C0008269,C0292818,C0674432,C0010286,C0020336,C0254211,C0700447,C0046462,C0030054,C0003751&scopes=umls)
API 14.3 SEMMED Chemical API: 0 hits
API 8.1: https://translator.broadinstitute.org/molepro_reasoner/query (POST -d {'message': {'query_graph': {'edges': [{'id': 'e00', 'source_id': 'n00', 'target_id': 'n01', 'type': 'related_to'}], 'nodes': [{'curie': 'MONDO:0020753', 'id': 'n00', 'type': 'disease'}, {'id': 'n01', 'type': 'gene'}]}}})
API 8.1 Molecular Data Provider A

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 297, in callSingleAPI
    result = tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/__init__.py", line 49, in transform
    return self.tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/base_transformer.py", line 78, in transform
    item = self.wrap(item)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/cord_transformer.py", line 16, in wrap
    for k in item.keys():
RuntimeError: dictionary keys changed during iteration


...

API 8.210: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:728/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.210 BioLink API: 0 hits
API 8.211: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1154/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.211 BioLink API: 0 hits
API 6.214: https://automat.renci.org/pharos/gene/disease/NCBIGene:1026
API 6.214 Automat PHAROS API: 1 hits
API 1.214: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:760
API 1.214 Automat CORD19 Scibite API: 1 hits
API 8.212: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1026/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.212 BioLink API: 1 hits
API 8.213: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:760/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.213 BioLink API: 34 hits
API 8.214: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3930/phenotypes?direct=True&rows=200&unselect

API 8.228 BioLink API: 25 hits
API 8.229: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:100/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.229 BioLink API: 67 hits
API 8.230: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:6347/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.230 BioLink API: 0 hits
API 8.231: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:6348/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.231 BioLink API: 0 hits
API 1.230: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:6347
API 1.230 Automat CORD19 Scibite API: 22 hits
API 8.232: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1234/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.232 BioLink API: 0 hits
API 8.233: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:6364/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.233 BioLink API: 0 hits
API 6.230: https://automat.re

API 3.245 Automat CORD19 Scigraph API: 0 hits
API 6.248: https://automat.renci.org/pharos/gene/disease/NCBIGene:3458
API 6.248 Automat PHAROS API: 13 hits
API 1.249: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:3458
API 1.249 Automat CORD19 Scibite API: 37 hits
API 3.246: https://automat.renci.org/cord19-scigraph/gene/disease/NCBIGene:1161
API 3.246 Automat CORD19 Scigraph API: 0 hits
API 6.249: https://automat.renci.org/pharos/gene/disease/NCBIGene:1161
API 6.249 Automat PHAROS API: 1 hits
API 3.247: https://automat.renci.org/cord19-scigraph/gene/disease/NCBIGene:64689
API 3.247 Automat CORD19 Scigraph API: 0 hits
API 3.248: https://automat.renci.org/cord19-scigraph/gene/disease/NCBIGene:9402
API 3.248 Automat CORD19 Scigraph API: 0 hits
API 3.249: https://automat.renci.org/cord19-scigraph/gene/disease/NCBIGene:3458
API 3.249 Automat CORD19 Scigraph API: 13 hits
API 8.248: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3313/phenotypes?direct=True&rows=

API 8.261 BioLink API: 74 hits
API 8.262: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:8878/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.262 BioLink API: 132 hits
API 8.263: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:5290/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.263: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:7040/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.263 BioLink API: 199 hits
API 8.264 BioLink API: 103 hits
API 8.265: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:7098/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.265 BioLink API: 0 hits
API 6.265: https://automat.renci.org/pharos/gene/disease/NCBIGene:8795
API 6.265 Automat PHAROS API: 1 hits
API 6.266: https://automat.renci.org/pharos/gene/disease/NCBIGene:8600
API 6.266 Automat PHAROS API: 2 hits
API 8.266: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:8795/phenotypes?direct

API 8.280: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:6962/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.280 BioLink API: 0 hits
API 1.280: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:6776
API 1.280 Automat CORD19 Scibite API: 3 hits
API 8.281: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:6776/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.281 BioLink API: 0 hits
API 1.281: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:6962
API 1.281 Automat CORD19 Scibite API: 0 hits
API 1.282: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:5781
API 1.282 Automat CORD19 Scibite API: 1 hits
API 1.283: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:6778
API 1.283 Automat CORD19 Scibite API: 5 hits
API 6.280: https://automat.renci.org/pharos/gene/disease/NCBIGene:6776
API 6.280 Automat PHAROS API: 1 hits
API 6.281: https://automat.renci.org/pharos/gene/disease/NCBIGene:5788

In [13]:
df2 = pd2.display_table_view()
df2.shape

(8353054, 27)

In [14]:
# filter gene -> symptoms table to only include rows where symptoms match disease symptoms
indices_with_symptom_outputs = [i for i, val in enumerate([x.upper() for x in list(df2['output_label'])]) if val in disease_symptoms]
indirect_relevant_genes_to_symptoms_df = df2.iloc[indices_with_symptom_outputs]
indirect_relevant_genes_to_symptoms_df.shape


(16057, 27)

In [15]:
list(dict.fromkeys(list(indirect_relevant_genes_to_symptoms_df['node2_label'])))

['ESR1',
 'UMLS:C0033551',
 'UMLS:C3642141',
 'UMLS:C1705556',
 'UMLS:C0014442',
 'UMLS:C0017337',
 'UMLS:C1305923',
 'EGF',
 'CAT',
 'AGTR1',
 'INS',
 'UMLS:C3539645',
 'UMLS:C1705633',
 'ACE',
 'ANG',
 'UMLS:C0033634',
 'UMLS:C1705543',
 'UMLS:C1709136',
 'TGFB1',
 'UMLS:C0669365',
 'APP',
 'ERVK-10',
 'UMLS:C0030956',
 'MMP2',
 'UMLS:C0010531',
 'UMLS:C0034861',
 'UMLS:C0600388',
 'UMLS:C1456380',
 'CAMP',
 'ATM',
 'TNF',
 'TP53',
 'MMP9',
 'UMLS:C1705997',
 'FOS',
 'ALB',
 'ADA',
 'UMLS:C1704947',
 'IFNA1',
 'LEP',
 'UMLS:C1706879',
 'UMLS:C1709820',
 'CA2',
 'UMLS:C0387583',
 'BAX',
 'GPI',
 'UMLS:C3539722',
 'UMLS:C2985367',
 'SQSTM1',
 'UMLS:C0002059',
 'UMLS:C0031727',
 'UMLS:C0033640',
 'UMLS:C0132555',
 'UMLS:C0208355',
 'UMLS:C0669372',
 'CISH',
 'CREB1',
 'GRK2',
 'UMLS:C1706384',
 'UMLS:C3540579',
 'POMC',
 'HLA-E',
 'STAT3',
 'UMLS:C0085940',
 'CD4',
 'UMLS:C0030016',
 'UMLS:C0282641',
 'UMLS:C1334043',
 'IGHE',
 'IL2',
 'UMLS:C1705581',
 'PIK3CA',
 'PPARA',
 'PTH',
 'SER

In [16]:
indirect_genes = 	list(indirect_relevant_genes_to_symptoms_df["node2_label"])
indirect_gene_dict = {x:indirect_genes.count(x) for x in indirect_genes}
indirect_gene_dict = dict(sorted(indirect_gene_dict.items(), key=lambda x: x[1], reverse=True)[:100])
indirect_gene_dict

{'TNF': 2336,
 'UMLS:C0017337': 999,
 'IFNA1': 670,
 'UMLS:C0030956': 504,
 'VEGFA': 498,
 'TGFB1': 476,
 'IL6': 456,
 'INS': 414,
 'CD4': 402,
 'FOS': 371,
 'IL2': 350,
 'TP53': 336,
 'UMLS:C0014442': 332,
 'UMLS:C3889436': 301,
 'POMC': 287,
 'UMLS:C3539722': 220,
 'UMLS:C0669372': 212,
 'CAMP': 198,
 'UMLS:C1705581': 196,
 'CAT': 195,
 'LEP': 184,
 'PTH': 180,
 'CA2': 168,
 'UMLS:C0031727': 165,
 'TLR4': 162,
 'MYD88': 162,
 'ATM': 138,
 'UMLS:C1705556': 135,
 'PIK3CA': 132,
 'UMLS:C1705744': 132,
 'UMLS:C0033634': 120,
 'STAT3': 120,
 'ACE': 117,
 'UMLS:C0033640': 117,
 'ABCB1': 116,
 'UMLS:C1706321': 111,
 'CRP': 110,
 'GRK2': 108,
 'APP': 96,
 'UMLS:C1706438': 92,
 'UMLS:C1367471': 92,
 'UMLS:C1709136': 90,
 'UMLS:C0669365': 90,
 'UMLS:C1706384': 90,
 'UMLS:C0033551': 87,
 'FAS': 84,
 'CCR5': 82,
 'UMLS:C1708411': 82,
 'UMLS:C1709820': 81,
 'UMLS:C0085940': 81,
 'UMLS:C1334043': 74,
 'UMLS:C1704947': 72,
 'UMLS:C1708431': 72,
 'UMLS:C1706005': 72,
 'CD79A': 69,
 'TNFRSF1A': 63,
 

## Top Genes -> Pathways 

In [17]:
# get gene inputs through hint module
gene_inputs = []
gene_names = list(direct_gene_dict.keys()) + list(indirect_gene_dict.keys())
gene_names = [x for x in gene_names if 'UMLS' not in x ]
gene_names = list(dict.fromkeys(gene_names))
gene_names
for gene in gene_names: 
    try: 
        gene_input = ht.query(gene)["Gene"][0]
        gene_inputs.append(gene_input)
    except: 
        print(gene + ' Failed')

# print(gene_inputs)

APP Failed


In [18]:
# pd3 = Predict(
#     input_objs=gene_inputs,
#     intermediate_nodes=[None], 
#     output_types =['Pathway'], 
#     config={"expand": False}
# )
# pd3.connect(verbose=True)

In [19]:
# df3 = pd3.display_table_view()
# df3.shape

In [20]:
# predict_many funciton will be used to run many BTE queries and return results as a single table
def predict_many(input_object_list, output_type_list, intermediate_node_list = ''):
    df_list = []
    for input_object in input_object_list: 
        for output_type in output_type_list: 
            if(len(intermediate_node_list) > 0):
                for inter in intermediate_node_list:
                    try: 
                        print("Running: " + input_object['name'] + ' --> intermediate type ' + inter + ' --> output type ' + output_type )
                        fc = FindConnection(input_obj=input_object, output_obj=output_type, intermediate_nodes=[inter])
                        fc.connect(verbose=False)
                        df = fc.display_table_view()
                        rows = df.shape[0]
                        if(rows > 0):
                            df_list.append(df)
                    except:
                        print(input_object['name'] + ' --> intermediate type ' + inter + ' --> output type ' + output_type + ' FAILED')
            else:
                try:
                    print("Running: " + input_object['name'] + ' --> output type ' + output_type )
                    fc = FindConnection(input_obj=input_object, output_obj=output_type, intermediate_nodes=None)
                    fc.connect(verbose=False)
                    df = fc.display_table_view()
                    rows = df.shape[0]
                    if(rows > 0):
                        df_list.append(df)
                except:
                    print(input_object['name'] + ' --> output type ' + output_type + ' FAILED')

    if(len(df_list) > 0):
        return pandas.concat(df_list)
    else:
        return None

In [21]:
gene_to_pathways = predict_many(gene_inputs, ['Pathway'])
list(dict.fromkeys(list(gene_to_pathways["output_name"])))

Running: tumor necrosis factor --> output type Pathway
Running: C-reactive protein --> output type Pathway
Running: interferon alpha 1 --> output type Pathway
Running: CD4 molecule --> output type Pathway
Running: interleukin 6 --> output type Pathway
Running: interleukin 2 --> output type Pathway
Running: angiotensin I converting enzyme --> output type Pathway
Running: methionyl-tRNA synthetase 1 --> output type Pathway
Running: C-C motif chemokine ligand 2 --> output type Pathway
Running: coagulation factor II, thrombin --> output type Pathway
Running: interferon gamma --> output type Pathway
Running: interleukin 1 beta --> output type Pathway
Running: coagulation factor III, tissue factor --> output type Pathway
Running: troponin I3, cardiac type --> output type Pathway
Running: glutamic--pyruvic transaminase --> output type Pathway
Running: myoglobin --> output type Pathway
Running: CD8a molecule --> output type Pathway
Running: cytochrome p450 oxidoreductase --> output type Pathwa

['DEVELOPMENTAL BIOLOGY',
 'CYTOKINE SIGNALING IN IMMUNE SYSTEM',
 'SIGNAL TRANSDUCTION',
 'IMMUNE SYSTEM',
 'TRANSCRIPTIONAL REGULATION OF WHITE ADIPOCYTE DIFFERENTIATION',
 'SIGNALING BY INTERLEUKINS',
 'TNFR1-INDUCED PROAPOPTOTIC SIGNALING',
 'REGULATION OF TNFR1 SIGNALING',
 'TNFR1-INDUCED NFKAPPAB SIGNALING PATHWAY',
 'TNFR1-MEDIATED CERAMIDE PRODUCTION',
 'TNFR2 NON-CANONICAL NF-KB PATHWAY',
 'INTERLEUKIN-10 SIGNALING',
 'INTERLEUKIN-4 AND INTERLEUKIN-13 SIGNALING',
 'DEATH RECEPTOR SIGNALLING',
 'TNF SIGNALING',
 'MATRIX METALLOPROTEINASES',
 'REGULATION OF TOLL-LIKE RECEPTOR SIGNALING PATHWAY',
 'SELENIUM MICRONUTRIENT NETWORK',
 'VITAMIN B12 METABOLISM',
 'MICRORNAS IN CARDIOMYOCYTE HYPERTROPHY',
 'TYPE II DIABETES MELLITUS',
 'FOLATE METABOLISM',
 'TNF RELATED WEAK INDUCER OF APOPTOSIS (TWEAK) SIGNALING PATHWAY',
 'ALZHEIMERS DISEASE',
 'TNF ALPHA SIGNALING PATHWAY',
 'ALLOGRAFT REJECTION',
 'ADIPOGENESIS',
 'SPINAL CORD INJURY',
 'AMYOTROPHIC LATERAL SCLEROSIS (ALS)',
 'APOP

In [22]:
# pathways
gene_to_pathway_results = {}
gene_to_pathway_genes = list(gene_to_pathways["output_name"]) # create list of genes
gene_to_pathway_genes = list(dict.fromkeys(gene_to_pathway_genes))  # remove duplicates

for gene in gene_to_pathway_genes: 
    gene_to_pathway_results[gene] = {
        'pathway_count' : 0,
        "genes_related" : []
    }

for index, row in gene_to_pathways.iterrows():
    gene_to_pathway_results[row['output_name']]['pathway_count'] = gene_to_pathway_results[row['output_name']]['pathway_count'] + 1
    gene_to_pathway_results[row['output_name']]['genes_related'].append(row['input'])
    

gene_to_pathway_results = dict(sorted(gene_to_pathway_results.items(), key = lambda x: x[1]['pathway_count'], reverse = True))

    
# gene_to_pathway_results
pandas.DataFrame.from_dict(gene_to_pathway_results, orient='index').iloc[0:50]

,pathway_count,genes_related
IMMUNE SYSTEM,40,"[TNF, CRP, IFNA1, CD4, IL6, IL2, CCL2, F2, IFN..."
SIGNAL TRANSDUCTION,33,"[TNF, IL6, IL2, CCL2, F2, F3, VEGFA, TGFB1, IN..."
CYTOKINE SIGNALING IN IMMUNE SYSTEM,29,"[TNF, IFNA1, CD4, IL6, IL2, CCL2, IFNG, IL1B, ..."
SIGNALING BY INTERLEUKINS,25,"[TNF, CD4, IL6, IL2, CCL2, IFNG, IL1B, VEGFA, ..."
INNATE IMMUNE SYSTEM,19,"[CRP, IFNA1, CD4, F2, IL1B, FOS, CAMP, CAT, TL..."
GENERIC TRANSCRIPTION PATHWAY,19,"[IL6, IL2, IFNG, VEGFA, TGFB1, INS, FOS, TP53,..."
RNA POLYMERASE II TRANSCRIPTION,19,"[IL6, IL2, IFNG, VEGFA, TGFB1, INS, FOS, TP53,..."
GENE EXPRESSION (TRANSCRIPTION),19,"[IL6, IL2, IFNG, VEGFA, TGFB1, INS, FOS, TP53,..."
DISEASE,18,"[CD4, IL6, F2, IL1B, VEGFA, TGFB1, POMC, PTH, ..."
METABOLISM OF PROTEINS,16,"[IL6, ACE, MARS1, CCL2, F2, ALB, TGFB1, INS, T..."


## Genes --> Bioprocesses

In [23]:
# bioprocesss
gene_to_bioprocesses = predict_many(gene_inputs, ['BiologicalProcess'])
gene_to_bioprocess_results = {}
gene_to_bioprocess_genes = list(gene_to_bioprocesses["output_name"]) # create list of genes
gene_to_bioprocess_genes = list(dict.fromkeys(gene_to_bioprocess_genes))  # remove duplicates

for gene in gene_to_bioprocess_genes: 
    gene_to_bioprocess_results[gene] = {
        'bioprocess_count' : 0,
        "genes_related" : []
    }

for index, row in gene_to_bioprocesses.iterrows():
    gene_to_bioprocess_results[row['output_name']]['bioprocess_count'] = gene_to_bioprocess_results[row['output_name']]['bioprocess_count'] + 1
    gene_to_bioprocess_results[row['output_name']]['genes_related'].append(row['input'])


Running: tumor necrosis factor --> output type BiologicalProcess
Running: C-reactive protein --> output type BiologicalProcess
Running: interferon alpha 1 --> output type BiologicalProcess
Running: CD4 molecule --> output type BiologicalProcess
Running: interleukin 6 --> output type BiologicalProcess
Running: interleukin 2 --> output type BiologicalProcess
Running: angiotensin I converting enzyme --> output type BiologicalProcess
Running: methionyl-tRNA synthetase 1 --> output type BiologicalProcess
Running: C-C motif chemokine ligand 2 --> output type BiologicalProcess
Running: coagulation factor II, thrombin --> output type BiologicalProcess
Running: interferon gamma --> output type BiologicalProcess
Running: interleukin 1 beta --> output type BiologicalProcess
Running: coagulation factor III, tissue factor --> output type BiologicalProcess
Running: troponin I3, cardiac type --> output type BiologicalProcess
Running: glutamic--pyruvic transaminase --> output type BiologicalProcess
Ru

In [24]:
gene_to_bioprocess_results = dict(sorted(gene_to_bioprocess_results.items(), key = lambda x: x[1]['bioprocess_count'], reverse = True))

In [28]:
counter = 0 
gene_to_bioprocess_results_copy = gene_to_bioprocess_results

for key in gene_to_bioprocess_results_copy.keys(): 
    if counter < 100: 
        if(('C0' in key) or ('C1' in key)): 
            try: 
                name = ht.query(key)['BiologicalProcess'][0]['name']
                gene_to_bioprocess_results[name] = gene_to_bioprocess_results[key]
                del gene_to_bioprocess_results[key]
            except: 
                del gene_to_bioprocess_results[key]
    counter = counter + 1

RuntimeError: dictionary keys changed during iteration

In [29]:
# gene_to_bioprocess_results
pandas.DataFrame.from_dict(gene_to_bioprocess_results, orient='index').iloc[0:50]

,bioprocess_count,genes_related
GROWTH,47,"[TNF, CRP, IFNA1, CD4, IL6, IL2, CCL2, F2, IFN..."
INFLAMMATION,47,"[TNF, TNF, CRP, CRP, IFNA1, CD4, IL6, IL6, IL2..."
GENE EXPRESSION,46,"[TNF, CRP, IFNA1, CD4, IL6, IL2, IL2, ACE, CCL..."
BREAKDOWN,40,"[TNF, IFNA1, CD4, IL6, IL2, ACE, CCL2, F2, IL1..."
ACTIVATION OF GLOBAL TRANSCRIPTION FROM RNA POLYMERASE II PROMOTER,40,"[TNF, TNF, TNF, TNF, IL6, IL6, IL2, IL1B, VEGF..."
IMMUNE RESPONSE,37,"[TNF, CRP, IFNA1, CD4, CD4, IL6, IL2, IL2, CCL..."
SECRETION,37,"[TNF, IFNA1, CD4, IL6, IL2, CCL2, F2, IFNG, IL..."
ANGIOGENESIS,36,"[TNF, IFNA1, CD4, IL6, IL2, CCL2, CCL2, F2, MB..."
CATAGEN,35,"[TNF, IFNA1, CD4, IL6, MARS1, CCL2, F2, IFNG, ..."
INNATE IMMUNE RESPONSE,34,"[TNF, CRP, CRP, IFNA1, CD4, IL6, IL2, CCL2, IL..."


## Genes --> AnatomicalEntity

In [35]:
# anatomical entity
gene_to_anatomical_entity = predict_many(gene_inputs, ['AnatomicalEntity'])
gene_to_anatomical_entity_results = {}
gene_to_anatomical_entity_genes = list(gene_to_anatomical_entity["output_name"]) # create list of genes
gene_to_anatomical_entity_genes = list(dict.fromkeys(gene_to_anatomical_entity_genes))  # remove duplicates

for gene in gene_to_anatomical_entity_genes: 
    gene_to_anatomical_entity_results[gene] = {
        'AE_count' : 0,
        "genes_related" : []
    }

for index, row in gene_to_anatomical_entity.iterrows():
    gene_to_anatomical_entity_results[row['output_name']]['AE_count'] = gene_to_anatomical_entity_results[row['output_name']]['AE_count'] + 1
    gene_to_anatomical_entity_results[row['output_name']]['genes_related'].append(row['input'])


Running: tumor necrosis factor --> output type AnatomicalEntity
Running: C-reactive protein --> output type AnatomicalEntity
Running: interferon alpha 1 --> output type AnatomicalEntity
Running: CD4 molecule --> output type AnatomicalEntity
Running: interleukin 6 --> output type AnatomicalEntity
Running: interleukin 2 --> output type AnatomicalEntity
Running: angiotensin I converting enzyme --> output type AnatomicalEntity
Running: methionyl-tRNA synthetase 1 --> output type AnatomicalEntity
Running: C-C motif chemokine ligand 2 --> output type AnatomicalEntity
Running: coagulation factor II, thrombin --> output type AnatomicalEntity
Running: interferon gamma --> output type AnatomicalEntity
Running: interleukin 1 beta --> output type AnatomicalEntity
Running: coagulation factor III, tissue factor --> output type AnatomicalEntity
Running: troponin I3, cardiac type --> output type AnatomicalEntity
Running: glutamic--pyruvic transaminase --> output type AnatomicalEntity
Running: myoglobi

In [39]:
gene_to_ae_results = dict(sorted(gene_to_anatomical_entity_results.items(), key = lambda x: x[1]['AE_count'], reverse = True))

In [42]:
pandas.DataFrame.from_dict(gene_to_ae_results, orient='index').iloc[50:100]

,AE_count,genes_related
GYRUS CENTRALIS POSTERIOR,16,"[CCL2, IL1B, TGFB1, TP53, CCR5, ANG, MMP2, C5A..."
CAUDAL MESENTERIC ARTERY,15,"[TNF, IFNA1, CD4, IL6, IL2, CCL2, MB, FOS, CAT..."
ENDOTHELIUM,15,"[TNF, CD4, IL6, ACE, ALB, VEGFA, INS, CA2, FAS..."
BRONCHI EPITHELIAL TISSUE,15,"[TNF, IL6, TNNI3, ALB, TP53, TLR4, ATM, PIK3CA..."
LEFT TESTICLE,15,"[CRP, IFNA1, CD4, F2, IL1B, CD8A, INS, CAT, PT..."
COLLUM,15,"[CRP, IFNA1, ALB, TP53, POMC, LEP, PIK3CA, STA..."
MUSCLE TISSUE,15,"[CD4, CCL2, ALB, TGFB1, INS, FOS, ATM, IRF3, C..."
EPIPHARYNX,15,"[MARS1, IL1B, CD8A, SON, FOS, CAT, TLR4, MYD88..."
BLOOD VESSEL INVOLUNTARY MUSCLE,14,"[TNF, CRP, IL6, F3, VEGFA, TGFB1, INS, TP53, C..."
THYMUS,14,"[TNF, IFNA1, CD4, IL2, CCL2, CD8A, INS, POMC, ..."


In [38]:
 gene_to_anatomical_entity_results

{'C0003964': {'AE_count': 6,
  'genes_related': ['TNF', 'IL6', 'VEGFA', 'VEGFA', 'TGFB1', 'LEP']},
 'UBERON:3010224': {'AE_count': 6,
  'genes_related': ['TNF', 'CRP', 'CAT', 'CA2', 'MYD88', 'PIK3CA']},
 'BLOOD': {'AE_count': 94,
  'genes_related': ['TNF',
   'TNF',
   'TNF',
   'CRP',
   'IFNA1',
   'IFNA1',
   'IFNA1',
   'IFNA1',
   'CD4',
   'CD4',
   'CD4',
   'IL6',
   'IL2',
   'IL2',
   'IL2',
   'IL2',
   'ACE',
   'MARS1',
   'CCL2',
   'CCL2',
   'CCL2',
   'F2',
   'IFNG',
   'IL1B',
   'F3',
   'F3',
   'F3',
   'GPT',
   'MB',
   'CD8A',
   'CD8A',
   'ALB',
   'ALB',
   'VEGFA',
   'VEGFA',
   'TGFB1',
   'TGFB1',
   'INS',
   'INS',
   'INS',
   'INS',
   'TP53',
   'TP53',
   'POMC',
   'POMC',
   'CAMP',
   'CAMP',
   'CAT',
   'CAT',
   'CAT',
   'LEP',
   'LEP',
   'PTH',
   'PTH',
   'PTH',
   'CA2',
   'TLR4',
   'ATM',
   'PIK3CA',
   'STAT3',
   'ABCB1',
   'ABCB1',
   'GRK2',
   'FAS',
   'FAS',
   'CCR5',
   'CD79A',
   'IL3',
   'IL3',
   'SOD1',
   'SOD1',
 